In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import math
import os
import numpy.linalg as la
import tensorflow as tf
from tensorflow.contrib.rnn import RNNCell
import scipy.sparse as sp
import time

def normalized_adj(adj):
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    normalized_adj = adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()
    normalized_adj = normalized_adj.astype(np.float32)
    return normalized_adj
    
def sparse_to_tuple(mx):
    mx = mx.tocoo()
    coords = np.vstack((mx.row, mx.col)).transpose()
    L = tf.SparseTensor(coords, mx.data, mx.shape)
    return tf.sparse_reorder(L) 
    
def calculate_laplacian(adj, lambda_max=1):  
    adj = normalized_adj(adj + sp.eye(adj.shape[0]))
    adj = sp.csr_matrix(adj)
    adj = adj.astype(np.float32)
    return sparse_to_tuple(adj)
    
def weight_variable_glorot(input_dim, output_dim, name=""):
    init_range = np.sqrt(6.0 / (input_dim + output_dim))
    initial = tf.random_uniform([input_dim, output_dim], minval=-init_range,
                            maxval=init_range, dtype=tf.float32)

    return tf.Variable(initial,name=name)  


class tgcnCell(RNNCell):
    """Temporal Graph Convolutional Network """

    def call(self, inputs, **kwargs):
        pass

    def __init__(self, num_units, adj, num_nodes, input_size=None,
                 act=tf.nn.tanh, reuse=None):

        super(tgcnCell, self).__init__(_reuse=reuse)
        self._act = act
        self._nodes = num_nodes
        self._units = num_units
        self._adj = []
        self._adj.append(calculate_laplacian(adj))


    @property
    def state_size(self):
        return self._nodes * self._units

    @property
    def output_size(self):
        return self._units

    def __call__(self, inputs, state, scope=None):

        with tf.variable_scope(scope or "tgcn"):
            with tf.variable_scope("gates"):  
                value = tf.nn.sigmoid(
                    self._gc(inputs, state, 2 * self._units, bias=1.0, scope=scope))
                r, u = tf.split(value=value, num_or_size_splits=2, axis=1)
            with tf.variable_scope("candidate"):
                r_state = r * state
                c = self._act(self._gc(inputs, r_state, self._units, scope=scope))
            new_h = u * state + (1 - u) * c
        return new_h, new_h


    def _gc(self, inputs, state, output_size, bias=0.0, scope=None):
        ## inputs:(-1,num_nodes)
        inputs = tf.expand_dims(inputs, 2)
        ## state:(batch,num_node,gru_units)
        state = tf.reshape(state, (-1, self._nodes, self._units))
        ## concat
        x_s = tf.concat([inputs, state], axis=2)
        input_size = x_s.get_shape()[2].value
        ## (num_node,input_size,-1)
        x0 = tf.transpose(x_s, perm=[1, 2, 0])  
        x0 = tf.reshape(x0, shape=[self._nodes, -1])
        
        scope = tf.get_variable_scope()
        with tf.variable_scope(scope):
            for m in self._adj:
                x1 = tf.sparse_tensor_dense_matmul(m, x0)
#                print(x1)
            x = tf.reshape(x1, shape=[self._nodes, input_size,-1])
            x = tf.transpose(x,perm=[2,0,1])
            x = tf.reshape(x, shape=[-1, input_size])
            weights = tf.get_variable(
                'weights', [input_size, output_size], initializer=tf.contrib.layers.xavier_initializer())
            x = tf.matmul(x, weights)  # (batch_size * self._nodes, output_size)
            biases = tf.get_variable(
                "biases", [output_size], initializer=tf.constant_initializer(bias, dtype=tf.float32))
            x = tf.nn.bias_add(x, biases)
            x = tf.reshape(x, shape=[-1, self._nodes, output_size])
            x = tf.reshape(x, shape=[-1, self._nodes * output_size])
        return x

from sklearn.metrics import mean_squared_error, mean_absolute_error

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true1, y_pred1 = np.array(y_true).flatten(), np.array(y_pred).flatten()
    y_true1, y_pred1 = y_true1[np.where(y_true1 > 10)[0]], y_pred1[np.where(y_true1 > 10)[0]]
    return np.mean(np.abs((y_true1 - y_pred1) / y_true1)) * 100

def compute_me(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true[np.where(y_true > 10)[0]], y_pred[np.where(y_true > 10)[0]])
    return mae, rmse, mape

In [4]:
def TGCN(_X, _weights, _biases):
    ###
    cell_1 = tgcnCell(gru_units, adj, num_nodes=num_nodes)
    cell = tf.nn.rnn_cell.MultiRNNCell([cell_1], state_is_tuple=True)
    _X = tf.unstack(_X, axis=1)
    outputs, states = tf.nn.static_rnn(cell, _X, dtype=tf.float32)
    m = []
    for i in outputs:
        o = tf.reshape(i,shape=[-1,num_nodes,gru_units])
        o = tf.reshape(o,shape=[-1,gru_units])
        m.append(o)
    last_output = m[-1]
    output = tf.matmul(last_output, _weights['out']) + _biases['out']
    output = tf.reshape(output,shape=[-1,num_nodes,pre_len])
    output = tf.transpose(output, perm=[0,2,1])
    output = tf.reshape(output, shape=[-1,num_nodes])
    return output, m, states
        

## SZ

In [5]:
X_train_sz = np.load('./data/taxi_sz/X_train20.npy').astype('float32')
X_val_sz = np.load('./data/taxi_sz/X_val20.npy').astype('float32')
X_test_sz = np.load('./data/taxi_sz/X_test20.npy').astype('float32')
y_train_sz = np.load('./data/taxi_sz/y_train20.npy').astype('float32')
y_val_sz = np.load('./data/taxi_sz/y_val20.npy').astype('float32')
y_test_sz = np.load('./data/taxi_sz/y_test20.npy').astype('float32')

edges_in_sz = np.load('./data/taxi_sz/edges_in_sz.npy')
print(edges_in_sz.shape)

high_similar_poi_sz = pd.read_csv('./data/taxi_sz/high_similar_poi.csv')
high_similar_poi_sz.head(2)

print(X_train_sz.shape, y_train_sz.shape)

(654, 2)
(101, 3672, 24) (101, 3672)


In [6]:
X_train, X_val, X_test = X_train_sz.transpose(1,2,0), X_val_sz.transpose(1,2,0), X_test_sz.transpose(1,2,0)
y_train, y_val, y_test = y_train_sz.T[:,np.newaxis,:], y_val_sz.T[:,np.newaxis,:], y_test_sz.T[:,np.newaxis,:]
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)


(3672, 24, 101) (504, 24, 101) (504, 24, 101) (3672, 1, 101) (504, 1, 101) (504, 1, 101)


In [7]:
mean, std = np.mean(X_train), np.std(X_train)
X_train = (X_train - mean)/std
X_val = (X_val - mean)/std
X_test = (X_test - mean)/std


In [8]:
adj = np.zeros((101, 101))

for i in range(101): 
    adj[i,i] = 1
    
for i in range(len(edges_in_sz)):
    adj[edges_in_sz[i,0], edges_in_sz[i,1]] = 1
    
for i in range(len(high_similar_poi_sz)):
    adj[high_similar_poi_sz['i'].iloc[i], high_similar_poi_sz['j'].iloc[i]] = 1


In [9]:
###### Settings ######
seq_len = time_len = 24
output_dim = pre_len = 1
batch_size = 36     
num_nodes = 101
lr = 0.01     
training_epoch = 30
gru_units = 64    

In [10]:
time_start = time.time()
tf.reset_default_graph()

totalbatch = int(X_train.shape[0]/batch_size)
training_data_count = len(X_train)

###### placeholders ######
inputs = tf.placeholder(tf.float32, shape=[None, seq_len, num_nodes])
labels = tf.placeholder(tf.float32, shape=[None, pre_len, num_nodes])

# Graph weights
weights = {
    'out': tf.Variable(tf.random_normal([gru_units, pre_len], mean=1.0), name='weight_o')}
biases = {
    'out': tf.Variable(tf.random_normal([pre_len]),name='bias_o')}


pred,ttts,ttto = TGCN(inputs, weights, biases)

y_pred = pred
      

###### optimizer ######
lambda_loss = 0.0015
Lreg = lambda_loss * sum(tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables())
label = tf.reshape(labels, [-1,num_nodes])
##loss
loss = tf.reduce_mean(tf.nn.l2_loss(y_pred-label) + Lreg)
##rmse
error = tf.sqrt(tf.reduce_mean(tf.square(y_pred-label)))
optimizer = tf.train.AdamOptimizer(lr).minimize(loss)

###### Initialize session ######
variables = tf.global_variables()
saver = tf.train.Saver(tf.global_variables())  
sess = tf.Session()
sess.run(tf.global_variables_initializer())

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [46]:
x_axe,batch_loss,batch_rmse,batch_pred = [], [], [], []
test_loss,test_rmse,test_mae,test_mape,test_pred = [],[],[],[],[]
  
for epoch in range(training_epoch):
    for m in range(totalbatch):
        mini_batch = X_train[m * batch_size : (m+1) * batch_size]
        mini_label = y_train[m * batch_size : (m+1) * batch_size]
        _, loss1, rmse1, train_output = sess.run([optimizer, loss, error, y_pred],
                                                 feed_dict = {inputs:mini_batch, labels:mini_label})
        batch_loss.append(loss1)
        #batch_rmse.append(rmse1 * max_value)

     # Test completely at every epoch
    loss2, rmse2, test_output = sess.run([loss, error, y_pred],
                                         feed_dict = {inputs:X_test, labels:y_test})
    mae, rmse, mape = compute_me(test_label, test_output)
    
    test_loss.append(loss2)
    test_rmse.append(rmse)
    test_mae.append(mae)
    test_mape.append(mape)
    test_pred.append(test_output1)
    
    print('Iter:{}'.format(epoch),
          'test_mae:{:.4}'.format(mae),
          'test_rmse:{:.4}'.format(rmse),
          'test_mape:{:.4}'.format(mape))
        
time_end = time.time()
print(time_end-time_start,'s')


Iter:0 test_mae:20.87 test_rmse:31.88 test_mape:55.42
Iter:1 test_mae:18.88 test_rmse:29.48 test_mape:51.16
Iter:2 test_mae:17.85 test_rmse:27.95 test_mape:48.43
Iter:3 test_mae:16.91 test_rmse:26.56 test_mape:45.76
Iter:4 test_mae:16.13 test_rmse:25.12 test_mape:44.4
Iter:5 test_mae:14.91 test_rmse:23.09 test_mape:42.94
Iter:6 test_mae:13.79 test_rmse:21.07 test_mape:40.36
Iter:7 test_mae:12.87 test_rmse:19.35 test_mape:38.05
Iter:8 test_mae:11.94 test_rmse:17.85 test_mape:35.54
Iter:9 test_mae:11.07 test_rmse:16.67 test_mape:33.19
Iter:10 test_mae:10.65 test_rmse:15.94 test_mape:32.03
Iter:11 test_mae:10.14 test_rmse:15.25 test_mape:30.87
Iter:12 test_mae:9.775 test_rmse:14.69 test_mape:29.86
Iter:13 test_mae:9.595 test_rmse:14.45 test_mape:29.27
Iter:14 test_mae:9.312 test_rmse:14.04 test_mape:28.7
Iter:15 test_mae:9.183 test_rmse:13.75 test_mape:27.93
Iter:16 test_mae:8.894 test_rmse:13.33 test_mape:27.13
Iter:17 test_mae:8.878 test_rmse:13.27 test_mape:26.83
Iter:18 test_mae:8.633

In [13]:
x_axe,batch_loss,batch_rmse,batch_pred = [], [], [], []
test_loss,test_rmse,test_mae,test_mape,test_pred = [],[],[],[],[]
  
for epoch in range(3):
    for m in range(totalbatch):
        mini_batch = X_train[m * batch_size : (m+1) * batch_size]
        mini_label = y_train[m * batch_size : (m+1) * batch_size]
        _, loss1, rmse1, train_output = sess.run([optimizer, loss, error, y_pred],
                                                 feed_dict = {inputs:mini_batch, labels:mini_label})
        batch_loss.append(loss1)
        #batch_rmse.append(rmse1 * max_value)
    
    t0 = time.time()
     # Test completely at every epoch
    loss2, rmse2, test_output = sess.run([loss, error, y_pred],
                                         feed_dict = {inputs:X_test, labels:y_test})
    print('time cost: ', time.time() - t0)



time cost:  6.427638053894043
time cost:  6.379633903503418
time cost:  6.439267158508301


## TaxiNY

In [14]:
X_train_ny = np.load('./data/taxi_ny/X_train20.npy')
y_train_ny = np.load('./data/taxi_ny/y_train20.npy')
X_val_ny = np.load('./data/taxi_ny/X_val20.npy')
y_val_ny = np.load('./data/taxi_ny/y_val20.npy')
X_test_ny = np.load('./data/taxi_ny/X_test20.npy')
y_test_ny = np.load('./data/taxi_ny/y_test20.npy')

edges_in_ny = np.load('./data/taxi_ny/edges_manhattan.npy')
ny_high_similar_poi = pd.read_csv('./data/taxi_ny/high_similar_poi.csv')

X_train, X_val, X_test = X_train_ny.transpose(1,2,0), X_val_ny.transpose(1,2,0), X_test_ny.transpose(1,2,0)
y_train, y_val, y_test = y_train_ny.T[:,np.newaxis,:], y_val_ny.T[:,np.newaxis,:], y_test_ny.T[:,np.newaxis,:]
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

mean, std = np.mean(X_train), np.std(X_train)
X_train = (X_train - mean)/std
X_val = (X_val - mean)/std
X_test = (X_test - mean)/std

adj = np.zeros((63, 63))

for i in range(63): 
    adj[i,i] = 1
    
for i in range(len(edges_in_ny)):
    adj[edges_in_ny[i,0], edges_in_ny[i,1]] = 1
    
for i in range(len(ny_high_similar_poi)):
    adj[ny_high_similar_poi['i'].iloc[i], ny_high_similar_poi['j'].iloc[i]] = 1


(8856, 24, 63) (1008, 24, 63) (1008, 24, 63) (8856, 1, 63) (1008, 1, 63) (1008, 1, 63)


In [15]:
###### Settings ######
seq_len = time_len = 24
output_dim = pre_len = 1
batch_size = 72     
num_nodes = 63
lr = 0.01     
training_epoch = 25
gru_units = 64    

In [16]:
time_start = time.time()
tf.reset_default_graph()

totalbatch = int(X_train.shape[0]/batch_size)
training_data_count = len(X_train)
        
###### placeholders ######
inputs = tf.placeholder(tf.float32, shape=[None, seq_len, num_nodes])
labels = tf.placeholder(tf.float32, shape=[None, pre_len, num_nodes])

# Graph weights
weights = {
    'out': tf.Variable(tf.random_normal([gru_units, pre_len], mean=1.0), name='weight_o')}
biases = {
    'out': tf.Variable(tf.random_normal([pre_len]),name='bias_o')}


pred,ttts,ttto = TGCN(inputs, weights, biases)

y_pred = pred
      

###### optimizer ######
lambda_loss = 0.0015
Lreg = lambda_loss * sum(tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables())
label = tf.reshape(labels, [-1,num_nodes])
##loss
loss = tf.reduce_mean(tf.nn.l2_loss(y_pred-label) + Lreg)
##rmse
error = tf.sqrt(tf.reduce_mean(tf.square(y_pred-label)))
optimizer = tf.train.AdamOptimizer(lr).minimize(loss)

###### Initialize session ######
variables = tf.global_variables()
saver = tf.train.Saver(tf.global_variables())  
sess = tf.Session()
sess.run(tf.global_variables_initializer())


In [11]:
x_axe,batch_loss,batch_rmse,batch_pred = [], [], [], []
test_loss,test_rmse,test_mae,test_mape,test_pred = [],[],[],[],[]
  
for epoch in range(training_epoch):
    for m in range(totalbatch):
        mini_batch = X_train[m * batch_size : (m+1) * batch_size]
        mini_label = y_train[m * batch_size : (m+1) * batch_size]
        _, loss1, rmse1, train_output = sess.run([optimizer, loss, error, y_pred],
                                                 feed_dict = {inputs:mini_batch, labels:mini_label})
        batch_loss.append(loss1)

     # Test completely at every epoch
    loss2, rmse2, test_output = sess.run([loss, error, y_pred],
                                         feed_dict = {inputs:X_test, labels:y_test})
    
    test_label = np.reshape(y_test,[-1,num_nodes])
    mae, rmse, mape = compute_me(test_label, test_output)
    
    test_loss.append(loss2)
    test_rmse.append(rmse)
    test_mae.append(mae)
    test_mape.append(mape)
    test_pred.append(test_output)
    
    print('Iter:{}'.format(epoch),
          'test_mae:{:.4}'.format(mae),
          'test_rmse:{:.4}'.format(rmse),
          'test_mape:{:.4}'.format(mape))
        
time_end = time.time()
print(time_end-time_start,'s')


Iter:0 test_mae:9.746 test_rmse:15.7 test_mape:33.06
Iter:1 test_mae:9.817 test_rmse:15.07 test_mape:32.23
Iter:2 test_mae:8.022 test_rmse:12.76 test_mape:28.31
Iter:3 test_mae:6.925 test_rmse:11.33 test_mape:25.71
Iter:4 test_mae:6.482 test_rmse:10.66 test_mape:24.48
Iter:5 test_mae:6.119 test_rmse:9.994 test_mape:23.55
Iter:6 test_mae:5.931 test_rmse:9.691 test_mape:22.58
Iter:7 test_mae:5.875 test_rmse:9.548 test_mape:22.12
Iter:8 test_mae:5.855 test_rmse:9.49 test_mape:22.03
Iter:9 test_mae:5.788 test_rmse:9.389 test_mape:21.75
Iter:10 test_mae:5.766 test_rmse:9.327 test_mape:21.57
Iter:11 test_mae:5.735 test_rmse:9.243 test_mape:21.55
Iter:12 test_mae:5.741 test_rmse:9.206 test_mape:21.41
Iter:13 test_mae:5.768 test_rmse:9.214 test_mape:21.23
Iter:14 test_mae:5.75 test_rmse:9.198 test_mape:21.1
Iter:15 test_mae:5.73 test_rmse:9.187 test_mape:21.12
Iter:16 test_mae:5.736 test_rmse:9.2 test_mape:21.29
Iter:17 test_mae:5.756 test_rmse:9.226 test_mape:21.39
Iter:18 test_mae:5.767 test

In [ ]:
x_axe,batch_loss,batch_rmse,batch_pred = [], [], [], []
test_loss,test_rmse,test_mae,test_mape,test_pred = [],[],[],[],[]
  
for epoch in range(training_epoch):
    for m in range(totalbatch):
        mini_batch = X_train[m * batch_size : (m+1) * batch_size]
        mini_label = y_train[m * batch_size : (m+1) * batch_size]
        _, loss1, rmse1, train_output = sess.run([optimizer, loss, error, y_pred],
                                                 feed_dict = {inputs:mini_batch, labels:mini_label})
        batch_loss.append(loss1)

    t0 = time.time()
    loss2, rmse2, test_output = sess.run([loss, error, y_pred],
                                         feed_dict = {inputs:X_test, labels:y_test})
    print('time cost: ', time.time() - t0)